This notebook was an attempt to build on a starter ResNext model given by the AffectNet team.  However, we ran into implementation issues around batch size, cropping, general weak model performance, and many errors in completing the model runtime.

In [1]:
from __future__ import division, print_function, absolute_import

import h5py
import tflearn

TRAIN_H5 = 'train.h5'
VAL_H5 = 'val.h5'

#BATCH_SIZE = 256
#CROP_SIZE = 49 # too small, weak model (validation accuracy ~35%)

BATCH_SIZE = 4 # too large, get Output elements too large for GPU kernel
CROP_SIZE = 227

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
train_h5f = h5py.File(TRAIN_H5, 'r')
trainX = train_h5f['X']
trainY = train_h5f['Y']

val_h5f = h5py.File(VAL_H5, 'r')
valX = val_h5f['X']
valY = val_h5f['Y']

In [3]:
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
MEAN = [0.53990436 , 0.4405486  , 0.39328504] # already calculated, expedite processing
img_prep.add_featurewise_zero_center(per_channel=True, mean=MEAN)


# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([CROP_SIZE, CROP_SIZE], padding=4)

In [4]:
n = 5 # 32 layers
# Building Residual Network
net = tflearn.input_data(shape=[None, CROP_SIZE, CROP_SIZE, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.resnext_block(net, n, 16, 32)
net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 32, 32)
net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 64, 32)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [5]:
# Training
net = tflearn.fully_connected(net, 11, activation='softmax')
opt = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=opt,
                         loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model = tflearn.DNN(net, checkpoint_path='model_resnet_affectnet_full',
                    max_checkpoints=10, tensorboard_verbose=0,
                    clip_gradients=0.)

In [ ]:
model.fit(trainX, trainY, n_epoch=1, validation_set=(valX, valY),
          snapshot_epoch=False, snapshot_step=500,
          show_metric=True, batch_size=BATCH_SIZE, shuffle=True,
          run_id='resnet_affectnet_full')

Training Step: 279  | total loss: 1.74110 | time: 430.457s
| Momentum | epoch: 003 | loss: 1.74110 - acc: 0.2735 -- iter: 01108/96011


As you can see from the above, at the current rate, training one epoch would have taken an estimated 70 hours.